In [1]:
import logging
from winterapi import WinterAPI
from wintertoo.models import WinterRaDecToO, SpringRaDecToO

In [2]:
logging.getLogger("winterapi").setLevel("DEBUG")

In [3]:
winter = WinterAPI()

In [4]:
# Program details
program = "2024A000"

In [5]:
# Target details
ra_deg = 144.419513
dec_deg = 20.037579
# Target name is a useful field for querying data, so put something useful!
# If you use Skyportal, we strongly advise you to use that name
target_name = "ZTF25aclckzl"

In [6]:
# A ToO with the same dither combo for Y/J
# Note that you can specify the TOTAL exposure time in seconds! This will be divided across dithers.
# You can see the duration of each individual exposure below.
too_1 = WinterRaDecToO(
    ra_deg=ra_deg,
    dec_deg=dec_deg,
    n_dither=8,
    total_exposure_time=960.0,
    filters=["Y", "J"],
    target_name=target_name,
)
too_1

WinterRaDecToO(ra_deg=144.419513, dec_deg=20.037579, use_field_grid=False, filters=['Y', 'J'], target_priority=50.0, target_name='ZTF25aclckzl', total_exposure_time=960.0, n_dither=8, n_repetitions=1, dither_distance=90.0, start_time_mjd=np.float64(61061.94537127345), end_time_mjd=np.float64(61068.94537128066), max_airmass=2.0, use_best_detector=True, camera='winter', single_exposure_time=120.0)

In [7]:
# A second ToO, with a different dither number for Hs
too_2 = WinterRaDecToO(
    ra_deg=ra_deg,
    dec_deg=dec_deg,
    n_dither=15,
    total_exposure_time=960.0,
    filters=["Hs"],
    target_name=target_name,
)
too_2

WinterRaDecToO(ra_deg=144.419513, dec_deg=20.037579, use_field_grid=False, filters=['Hs'], target_priority=50.0, target_name='ZTF25aclckzl', total_exposure_time=960.0, n_dither=15, n_repetitions=1, dither_distance=90.0, start_time_mjd=np.float64(61061.94537137891), end_time_mjd=np.float64(61068.94537138248), max_airmass=2.0, use_best_detector=True, camera='winter', single_exposure_time=64.0)

In [8]:
too_list = [too_1, too_2]

In [9]:
program_list = winter.get_programs()
print(f"Available programs: {program_list}")
assert program in program_list, f"program {program} not found! Add this program first."

Available programs: ['2023A001', '2024A000', '2024A001', '2024A004']


In [10]:
# Schedule has three observations in total
local_schedule = winter.build_schedule_locally(program_name=program, data=too_list)
local_schedule

targName       raDeg     decDeg    fieldID filter  visitExpTime  \
0  ZTF25aclckzl  144.419513  20.037579  999999999      Y         960.0   
1  ZTF25aclckzl  144.419513  20.037579  999999999      J         960.0   
2  ZTF25aclckzl  144.419513  20.037579  999999999     Hs         960.0   

   singleExpTime  priority     progPI  progName  progID    validStart  \
0          120.0      50.0  MKasliwal  2024A000       3  61061.945371   
1          120.0      50.0  MKasliwal  2024A000       3  61061.945371   
2           64.0      50.0  MKasliwal  2024A000       3  61061.945371   

      validStop  observed  maxAirmass  ditherNumber  ditherStepSize  \
0  61068.945371     False         2.0             8            90.0   
1  61068.945371     False         2.0             8            90.0   
2  61068.945371     False         2.0            15            90.0   

   bestDetector  camera  obsHistID  
0          True  winter          0  
1          True  winter          1  
2          True  winter          2

In [11]:
# If submit_trigger is False, the API will just check the schedule but not put in a ToO
# Set this to True to actually trigger!
api_res, api_schedule = winter.submit_too(
    program_name=program, data=too_list, submit_trigger=False
)

In [12]:
# Let's see what message the API sent back
api_res.json()["msg"]

'Request passed validation! Triggering was set to False. Schedule was not saved to disk and has no assigned name.'

In [13]:
# This is the schedule sent back by the API. Reassuringly, it matches the local one
api_schedule

targName       raDeg     decDeg    fieldID filter  visitExpTime  \
0  ZTF25aclckzl  144.419513  20.037579  999999999      Y         960.0   
1  ZTF25aclckzl  144.419513  20.037579  999999999      J         960.0   
2  ZTF25aclckzl  144.419513  20.037579  999999999     Hs         960.0   

   singleExpTime  priority     progPI  progName  progID    validStart  \
0          120.0      50.0  MKasliwal  2024A000       3  61061.945371   
1          120.0      50.0  MKasliwal  2024A000       3  61061.945371   
2           64.0      50.0  MKasliwal  2024A000       3  61061.945371   

      validStop  observed  maxAirmass  ditherNumber  ditherStepSize  \
0  61068.945371     False         2.0             8            90.0   
1  61068.945371     False         2.0             8            90.0   
2  61068.945371     False         2.0            15            90.0   

   bestDetector  camera  obsHistID  
0          True  winter          0  
1          True  winter          1  
2          True  winter          2

In [14]:
# Uncomment to trigger for real
api_res, api_schedule = winter.submit_too(
    program_name=program, data=too_list, submit_trigger=True
)
api_res.json()["msg"]

"Request passed validation! Triggering was set to True. Schedule name is 'request_2024A000_2026_01_21_14_42_19' ."

In [15]:
# This is the schedule sent back by the API. Reassuringly, it matches the local one
api_schedule

targName       raDeg     decDeg    fieldID filter  visitExpTime  \
0  ZTF25aclckzl  144.419513  20.037579  999999999      Y         960.0   
1  ZTF25aclckzl  144.419513  20.037579  999999999      J         960.0   
2  ZTF25aclckzl  144.419513  20.037579  999999999     Hs         960.0   

   singleExpTime  priority     progPI  progName  progID    validStart  \
0          120.0      50.0  MKasliwal  2024A000       3  61061.945371   
1          120.0      50.0  MKasliwal  2024A000       3  61061.945371   
2           64.0      50.0  MKasliwal  2024A000       3  61061.945371   

      validStop  observed  maxAirmass  ditherNumber  ditherStepSize  \
0  61068.945371     False         2.0             8            90.0   
1  61068.945371     False         2.0             8            90.0   
2  61068.945371     False         2.0            15            90.0   

   bestDetector  camera  obsHistID  
0          True  winter          0  
1          True  winter          1  
2          True  winter          2

# Submitting SPRING ToO requests

You can also submit ToOs to SPRING. The procedure is the same, though there is no field grid so the only requests are RA/Dec ones. The default exposure times / number of dithers are different for SPRING, but you can still customise these if you like as you can for WINTER.

In [16]:
too_1 = SpringRaDecToO(
    ra_deg=ra_deg, dec_deg=dec_deg, filters=["J"], target_name=target_name
)
too_1

SpringRaDecToO(ra_deg=144.419513, dec_deg=20.037579, use_field_grid=False, filters=['J'], target_priority=50.0, target_name='ZTF25aclckzl', total_exposure_time=900.0, n_dither=30, n_repetitions=1, dither_distance=60.0, start_time_mjd=np.float64(61061.9460627211), end_time_mjd=np.float64(61068.94606272622), max_airmass=2.0, use_best_detector=True, camera='spring', single_exposure_time=30.0)

In [17]:
too_list = [too_1]

In [18]:
# Schedule has three observations in total
local_schedule = winter.build_schedule_locally(program_name=program, data=too_list)
local_schedule

targName       raDeg     decDeg    fieldID filter  visitExpTime  \
0  ZTF25aclckzl  144.419513  20.037579  999999999      J         900.0   

   singleExpTime  priority     progPI  progName  progID    validStart  \
0           30.0      50.0  MKasliwal  2024A000       3  61061.946063   

      validStop  observed  maxAirmass  ditherNumber  ditherStepSize  \
0  61068.946063     False         2.0            30            60.0   

   bestDetector  camera  obsHistID  
0          True  spring          0

**Be careful to use the different submit function for SPRING ToOs!**

In [19]:
# If submit_trigger is False, the API will just check the schedule but not put in a ToO
# Set this to True to actually trigger!
api_res, api_schedule = winter.submit_too_spring(
    program_name=program, data=too_list, submit_trigger=False
)

In [20]:
# Let's see what message the API sent back
api_res.json()["msg"]

'Request passed validation! Triggering was set to False. Schedule was not saved to disk and has no assigned name.'

In [21]:
# This is the schedule sent back by the API. Reassuringly, it matches the local one
api_schedule

targName       raDeg     decDeg    fieldID filter  visitExpTime  \
0  ZTF25aclckzl  144.419513  20.037579  999999999      J         900.0   

   singleExpTime  priority     progPI  progName  progID    validStart  \
0           30.0      50.0  MKasliwal  2024A000       3  61061.946063   

      validStop  observed  maxAirmass  ditherNumber  ditherStepSize  \
0  61068.946063     False         2.0            30            60.0   

   bestDetector  camera  obsHistID  
0          True  spring          0

# Non-standard exposure times

If you submit a non-default exposure time for a target, whether via WINTER or SPRING, you must also request darks with the same exposure time. You do this by including the dark filter in your request.

In [22]:
too_1 = SpringRaDecToO(
    ra_deg=ra_deg,
    dec_deg=dec_deg,
    filters=["J", "dark"],
    target_name=target_name,
    n_dither=60,
)
too_1

SpringRaDecToO(ra_deg=144.419513, dec_deg=20.037579, use_field_grid=False, filters=['J', 'dark'], target_priority=50.0, target_name='ZTF25aclckzl', total_exposure_time=900.0, n_dither=60, n_repetitions=1, dither_distance=60.0, start_time_mjd=np.float64(61061.946349995094), end_time_mjd=np.float64(61068.9463499981), max_airmass=2.0, use_best_detector=True, camera='spring', single_exposure_time=15.0)

In [24]:
api_res, api_schedule = winter.submit_too_spring(
    program_name=program, data=too_list, submit_trigger=False
)

In [25]:
# Let's see what message the API sent back
api_res.json()["msg"]

'Request passed validation! Triggering was set to False. Schedule was not saved to disk and has no assigned name.'

In [26]:
# This is the schedule sent back by the API. Reassuringly, it matches the local one
api_schedule

targName       raDeg     decDeg    fieldID filter  visitExpTime  \
0  ZTF25aclckzl  144.419513  20.037579  999999999      J         900.0   

   singleExpTime  priority     progPI  progName  progID    validStart  \
0           30.0      50.0  MKasliwal  2024A000       3  61061.946063   

      validStop  observed  maxAirmass  ditherNumber  ditherStepSize  \
0  61068.946063     False         2.0            30            60.0   

   bestDetector  camera  obsHistID  
0          True  spring          0